In [2]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('soybean.csv')

In [4]:
data.shape

(683, 36)

In [5]:
pd.set_option('max_columns', 36)
data.head(10)

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,plant-growth,leaves,leafspots-halo,leafspots-marg,leafspot-size,leaf-shread,leaf-malf,leaf-mild,stem,lodging,stem-cankers,canker-lesion,fruiting-bodies,external-decay,mycelium,int-discolor,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,october,normal,gt-norm,norm,yes,same-lst-yr,low-areas,pot-severe,none,90-100,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,no,above-sec-nde,brown,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
1,august,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,severe,fungicide,80-89,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,yes,above-sec-nde,brown,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
2,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,fungicide,lt-80,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,yes,above-sec-nde,dna,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
3,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,none,80-89,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,yes,above-sec-nde,dna,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
4,october,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,pot-severe,none,lt-80,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,yes,above-sec-nde,brown,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
5,september,normal,gt-norm,norm,yes,same-lst-sev-yrs,scattered,pot-severe,none,80-89,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,yes,above-sec-nde,dna,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
6,september,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,pot-severe,fungicide,90-100,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,no,above-sec-nde,brown,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
7,august,normal,gt-norm,norm,no,same-lst-yr,scattered,pot-severe,none,lt-80,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,yes,above-sec-nde,brown,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
8,october,normal,gt-norm,norm,yes,same-lst-sev-yrs,scattered,pot-severe,fungicide,80-89,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,yes,above-sec-nde,brown,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
9,august,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,severe,none,lt-80,abnorm,abnorm,absent,dna,dna,absent,absent,absent,abnorm,yes,above-sec-nde,brown,present,firm-and-dry,absent,none,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker


In [6]:
data_copy = data.copy()

y_train = data_copy.pop('class')
x_train = data_copy

In [7]:
lb = LabelEncoder()

y_train = lb.fit_transform(y_train)

In [8]:
classes = data['class']

In [9]:
len(classes.unique())

19

In [10]:
data.isnull().count()

date               683
plant-stand        683
precip             683
temp               683
hail               683
crop-hist          683
area-damaged       683
severity           683
seed-tmt           683
germination        683
plant-growth       683
leaves             683
leafspots-halo     683
leafspots-marg     683
leafspot-size      683
leaf-shread        683
leaf-malf          683
leaf-mild          683
stem               683
lodging            683
stem-cankers       683
canker-lesion      683
fruiting-bodies    683
external-decay     683
mycelium           683
int-discolor       683
sclerotia          683
fruit-pods         683
fruit-spots        683
seed               683
mold-growth        683
seed-discolor      683
seed-size          683
shriveling         683
roots              683
class              683
dtype: int64

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, train_size=0.8)

In [13]:
feature_columns = []

for a in x_train.columns:
    vocabulary = x_train[a].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(a, vocabulary))

In [17]:
#Piece of code taken from Tensorflow documentation

def make_input_fn(data_df, label_df, num_epochs=3000, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

In [18]:
input_train = make_input_fn(x_train, y_train)
input_test = make_input_fn(x_test, y_test, num_epochs=1, shuffle=False)

In [19]:
model = tf.estimator.LinearClassifier(feature_columns=feature_columns, n_classes=19)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\rafae\\AppData\\Local\\Temp\\tmp8kemvt2i', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
model.train(input_train)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


C:\Users\rafae\anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\rafae\AppData\Local\Temp\tmp8kemvt2i\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.944439, step = 0
INFO:tensorflow:global_step/sec: 60.7259
INFO:tensorflow:loss = 0.22586286, step = 100 (1.636 sec)
INFO:tensorflow:global_step/sec: 236.479
INFO:tensorflow:loss = 0.106455564, step = 200 (0.421 sec)
INFO:tensorflow:global_step/sec: 238.732
INFO:tensorflow:loss = 0.1330866, step = 300 (0.420 sec)
INFO:tensorflow:global_step/sec: 206.616
INFO:tensorflow:loss = 0.076887324, step = 400

In [21]:
model.evaluate(input_test)

INFO:tensorflow:Calling model_fn.


C:\Users\rafae\anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-01-29T22:51:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\rafae\AppData\Local\Temp\tmp8kemvt2i\model.ckpt-540
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 2.23254s
INFO:tensorflow:Finished evaluation at 2022-01-29-22:51:29
INFO:tensorflow:Saving dict for global step 540: accuracy = 0.8978102, average_loss = 0.2664264, global_step = 540, loss = 0.32829928
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 540: C:\Users\rafae\AppData\Local\Temp\tmp8kemvt2i\model.ckpt-540


{'accuracy': 0.8978102,
 'average_loss': 0.2664264,
 'loss': 0.32829928,
 'global_step': 540}

In [53]:
def test(position, all=False):
    x = model.predict(make_input_fn(x_test[position:position + 1], y_test[position:position + 1], num_epochs=1, shuffle=False))
    print('\n\nExpected: ' + classes[y_test[position]])
    print('Predicted: ' + classes[np.argmax(x)])
    print('\n\n')

    a = list(x)[0]['probabilities']
    
    print('Probability: ' + str(np.max(a)))

    if all:
        print('\nAll probabilities: ' + str(a))

In [55]:
test(122, True)



Expected: diaporthe-stem-canker
Predicted: diaporthe-stem-canker



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\rafae\AppData\Local\Temp\tmp8kemvt2i\model.ckpt-540
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Probability: 0.997773

All probabilities: [9.0092917e-06 1.7102266e-06 9.9777299e-01 3.9307292e-06 7.7030621e-05
 6.0462285e-06 2.9357951e-05 3.8761995e-05 7.1524564e-06 1.0325379e-04
 2.7274693e-04 1.9808879e-04 5.3039403e-04 1.4906531e-05 1.3350225e-06
 6.6016504e-04 6.4434578e-05 1.3211185e-04 7.6601405e-05]
